## First, let's install required packages

In [1]:
!pip install git+https://github.com/huggingface/transformers.git

  Cloning https://github.com/huggingface/transformers.git to /tmp/pip-req-build-neug_5m3
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers.git /tmp/pip-req-build-neug_5m3
  Resolved https://github.com/huggingface/transformers.git to commit 0e0516c119a3d1a2342d09166e583bb6fa647c8a
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 32.0 MB/s eta 0:00:00a 0:00:01
  Created wheel for transformers: filename=transformers-4.48.0.dev0-py3-none-any.whl size=10322147 sha256=b5742746315547069777034b9c9308a66a436b0f1c35f821b56832f7f10d1fc0
  Stored in directory: /tmp/pip-ephem-wheel-cache-_w5psbev/wheels/e7/9c/5b/e1a9c8007c343041e61cc484433d512ea9274272e3fcbe7c16
Successfully built transformers
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.19.1
    Uninstalling tokenizers-0.19.1

In [2]:
!pip install pypdf langchain faiss-cpu torch sentence-transformers langchain-community

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.8/41.8 kB 110.0 kB/s eta 0:00:000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 17.9 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.5/27.5 MB 70.5 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 16.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 71.6 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 411.6/411.6 kB 22.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 326.4/326.4 kB 18.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.5/73.5 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.6/78.6 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 131.3/131.3 kB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.5/54.5 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 3.5 MB/s eta 0:00:00


In [23]:
!curl -fsSL https://ollama.com/install.sh | sh
import subprocess
process = subprocess.Popen("ollama serve", shell=True) #runs on a different thread
#Download model
!ollama pull llama3.2:3b
!pip install ollama
import ollama

/usr/lib/python3.10/pty.py:89: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid, fd = os.forkpty()


>>> Cleaning up old version at /usr/local/lib/ollama
>>> Installing ollama to /usr/local
>>> Downloading Linux amd64 bundle
############################################################################################# 100.0%#######                                                                           22.4%############################################################                        76.8%
>>> Adding ollama user to video group...
>>> Adding current user to ollama group...
>>> Creating ollama systemd service...
>>> The Ollama API is now available at 127.0.0.1:11434.
>>> Install complete. Run "ollama" from the command line.
pulling manifest ⠙ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠸ pulling manifest ⠼ pulling manifest 
pulling dde5aa3fc5ff...   0% ▕                ▏    0 B/2.0 GB                  pulling manifest 
pulling dde5aa3fc5ff...   0% ▕                ▏    0 B/2.0 GB                  pulling manifest 
pulling dde5aa3fc5ff...   1% ▕                ▏  22 MB/

## Import necessary libraries

In [4]:
import os
from pypdf import PdfReader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from sentence_transformers import SentenceTransformer
from typing import List, Union
import torch
from langchain.llms import Ollama
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.chains.question_answering import load_qa_chain

## Step 1: Load and extract text from PDF

In [5]:
##Let me write some helper function for pdf extraction
def extract_text_from_pdf(pdf_path):
    pdf_reader = PdfReader(pdf_path)
    text = ""
    for page in pdf_reader.pages:
        text += page.extract_text()
    return text

In [12]:
raw_text = extract_text_from_pdf("/kaggle/input/muthu-test/Muthu_Palaniappan_M_CV_Winter.pdf")
print(f"Length of extracted Text: {len(raw_text)}")

Length of extracted Text: 11023


In [13]:
print("\nFirst 1000 characters of text:")
print(raw_text[:1000])


First 1000 characters of text:
Muthu Palaniappan M Email: muthupalaniappan.9999@gmail.com
LinkedIn: https://www.linkedin.com Mobile: +91-9345763899
Github: github.com/MuthuPalaniappan925
Objective
Driven and ambitious Junior Data Scientist with a research-oriented background, seeking opportunities to enhance my
skill sets and actively contribute to the design and development of innovative solutions, leveraging the power of
cutting-edge machine learning algorithms and GenAI systems
Education
• Shiv Nadar University Chennai Chennai, India
B.Tech AI & Data Science; CGPA: 9.60 August 2021 - Present
Courses: Deep Learning, Machine Learning Techniques, Foundations of Data Science, Exploratory Data Analysis, Statistical Inference,
and Analysis of Algorithms.
Organizations: SNUC Coding Club, National Service Scheme
• Cauvery Global Senior Secondary School Trichy, India
CBSE SCORE: 475/500 July 2020 - May-2021
Achievements: School Science Stream Topper, Represented at KIIT-Tech Extravaganza
Sk

## Step 2: Split text into chunks
- https://python.langchain.com/docs/how_to/recursive_text_splitter/
- https://www.linkedin.com/pulse/text-splitting-techniques-explained-how-enhance-your-systems-prakash-v151c/
- https://github.com/FullStackRetrieval-com/RetrievalTutorials/blob/main/tutorials/LevelsOfTextSplitting/5_Levels_Of_Text_Splitting.ipynb

In [14]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=200,
    length_function=len
)

In [15]:
text_chunks = text_splitter.split_text(raw_text)
print(f"\nNumber of text chunks: {len(text_chunks)}")


Number of text chunks: 14


In [16]:
print("\nExample chunk:")
print(text_chunks[10])


Example chunk:
with a Recurrent Deep Neural Network model.
◦ Conference: IEEE ICITIIT 2024 Kerala Region
◦ Role: Co-AuthorResearch Contributions
• Utilizing PEBL Framework for Psychometric Battery Development: Exploring Cognitive Analysis and
Clustering: ὖ∠
◦ Research Focus: Explored human cognitive architecture using the PEBL framework, designed psychometric questions in
C++, developed reinforcement learning models.
◦ STRIS Grant Award: Recognized for excellence in research and received a grant of 18,500 rupees in support of the project.
• Computational Fluid Dynamics based tools to the aid of clinical decision making in the management of
intracranial anueurysms.: ὖ∠
◦ Contributions: Focused on the application of virtual coiling techniques, offering valuable insights into the treatment of
these vascular abnormalities.
◦ Guide: Dr. Jayanand Sudhir B, Head, Department of Neurosurgery, SCTIMST.
Industry Accreditation
• Artificial Intelligence Virtual Experience Program Cognizant


## Step 3: Initialize the embedding model
- https://huggingface.co/nomic-ai/modernbert-embed-base

### Create a custom embedding class to integrate with LangChain

In [17]:
class MBEmbeddings:
    def __init__(self, model_name: str = "nomic-ai/modernbert-embed-base"):
        """Initialize the embeddings class with a SentenceTransformer model.
        
        Args:
            model_name: Name or path of the SentenceTransformer model
        """
        if isinstance(model_name, str):
            self.model = SentenceTransformer(model_name)
        else:
            self.model = model_name
    
    def embed_documents(self, texts: List[str]) -> List[List[float]]:
        """Generate embeddings for a list of documents.
        
        Args:
            texts: List of texts to generate embeddings for
            
        Returns:
            List of embeddings, one per text
        """
        embeddings = self.model.encode(texts)
        return embeddings.tolist()
    
    def embed_query(self, text: str) -> List[float]:
        """Generate embedding for a single query text.
        
        Args:
            text: Query text to generate embedding for
            
        Returns:
            Embedding for the text
        """
        embedding = self.model.encode([text])[0]
        return embedding.tolist()
    
    def __call__(self, text: Union[str, List[str]]) -> List[Union[List[float], float]]:
        """Make the class callable to support both single texts and lists of texts.
        
        Args:
            text: Single text string or list of texts
            
        Returns:
            Embeddings for the input text(s)
        """
        if isinstance(text, str):
            return self.embed_query(text)
        return self.embed_documents(text)

embeddings = MBEmbeddings("nomic-ai/modernbert-embed-base")

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/210 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/445k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/54.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.26k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/596M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/20.8k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/3.58M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/694 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/296 [00:00<?, ?B/s]

## Step 4: Create vector store using FAISS

In [18]:
vectorstore = FAISS.from_texts(text_chunks, embeddings)

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

### Let's see some example for Similarity Search

In [19]:
query = "Research Works"  
similar_docs = vectorstore.similarity_search(query, k=2)
print("\nExample similarity search results:")
for i, doc in enumerate(similar_docs):
    print(f"\nResult {i+1}:")
    print(doc.page_content[:200] + "...")

Batches:   0%|          | 0/1 [00:00<?, ?it/s]


Example similarity search results:

Result 1:
embeddings and relationships between classes without direct training data.
◦ Conference: IEEE Tensymp 2023 Australia Region 10, Top 100 Paper.
◦ Role: Primary Author
• Beyond Convex Settings: Can PSO ...

Result 2:
merchant value for PayPal’s products (XO, P2P, Installments, BNPL, QRC, etc.) over 12 and 36 months, tailored for
core and growth market clusters, driving strategic decision-making for product adoptio...


## Step 5: Initialize LLM
- https://ollama.com
- https://github.com/ollama/ollama

In [24]:
llm = Ollama(model="llama3.2:3b")

## Step 6: Create QA Chain
- https://python.langchain.com/api_reference/langchain/chains/langchain.chains.retrieval_qa.base.RetrievalQA.html

In [25]:
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=vectorstore.as_retriever(search_kwargs={'k': 2})
)

In [29]:
question = "Give me Muthu email address"  #IITM
response = qa_chain.run(question)
print("\nExample QA Response:")
print(f"Question: {question}")
print(f"Answer: {response}")

Batches:   0%|          | 0/1 [00:00<?, ?it/s]


Example QA Response:
Question: Give me Muthu email address
Answer: muthupalaniappan.9999@gmail.com


In [31]:
question = "What Muthu did in PayPal during his internhsip?"  #IITM
response = qa_chain.run(question)
print("\nExample QA Response:")
print(f"Question: {question}")
print(f"Answer: {response}")

Batches:   0%|          | 0/1 [00:00<?, ?it/s]


Example QA Response:
Question: What Muthu did in PayPal during his internhsip?
Answer: According to the text, Muthu implemented an end-to-end data pipeline to calculate the incremental lift in merchant value for PayPal's products (XO, P2P, Installments, BNPL, QRC, etc.) over 12 and 36 months during his internship at PayPal Bengaluru, India.


## Now let's do it with custom prompt templates

In [52]:
qa_template = """You are a helpful AI assistant which help HR team that provides accurate, informative answers based on the given context only.
Use the following context to answer the question. If you cannot find the answer in the context, say "I don't have enough information to answer this question."

Context: 
{context}

Question: {question}


Answer:"""

## Ref
- https://python.langchain.com/api_reference/core/prompts/langchain_core.prompts.prompt.PromptTemplate.html

In [53]:
## Prompt Template
PROMPT = PromptTemplate(
    template=qa_template,
    input_variables=["context", "question"]
)

In [54]:
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=vectorstore.as_retriever(search_kwargs={'k': 2}),
    chain_type_kwargs={
        "prompt": PROMPT
    }
)

- llm: This is our language model that will generate the final answers. It processes the retrieved documents and question to produce responses.

- chain_type="stuff": This specifies how documents are combined before being sent to the LLM
("stuff" means all retrieved documents are simply concatenated into one context window
This is the simplest approach but works well when you're retrieving a small number of documents that can fit in the context window)

- retriever=vectorstore.as_retriever(search_kwargs={'k': 2})
(Converts your vector store into a retriever
k=2 means it will fetch the 2 most relevant documents for each query
The vector store uses similarity search to find these relevant documents)


- chain_type_kwargs={"prompt": PROMPT}(
Allows you to customize the prompt template
The prompt should include {context} and {question} variables
{context} will be filled with retrieved documents
{question} will be filled with the user's question)

In [55]:
## Helper Function to retrive answer
def response_generation(q):
    docs = vectorstore.similarity_search(question, k=2)
    context = "\n\n".join([doc.page_content for doc in docs])
    filled_prompt = PROMPT.format(context=context, question=question)
    response = qa_chain.run(q)

    print(f"\nQ: {q}")
    print(f"A: {response}")

In [56]:
response_generation("Give me Muthu email address")

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]


Q: Give me Muthu email address
A: Muthu Palaniappan's email address is muthupalaniappan.9999@gmail.com.


In [60]:
response_generation("Talk about his internship details at PayPal")

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]


Q: Talk about his internship details at PayPal
A: According to the context, here are the internship details of the individual at PayPal:

1. Internship Position: Data Analyst Intern
2. Duration: May 2024 - August 2024
3. Worked on a project related to Global Analytics and Decision Science, where he implemented an end-to-end data pipeline.
4. The project involved calculating the incremental lift in merchant value for PayPal's products (XO, P2P, Installments, BNPL, QRC, etc.) over 12 and 36 months.

Note that there is no mention of specific tasks or responsibilities during his internship at PayPal.
